In [ ]:
!pip install SpeechRecognition gTTS pygame python-vlc ipywidgets
!pip install --upgrade opencv-python opencv-contrib-python


In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from time import sleep
from tkinter import filedialog
import numpy as np
import vlc
import glob
import cv2
import time
import os
import speech_recognition as sr
from IPython.display import display
from gtts import gTTS
from pygame import mixer

class AuthenticationApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Authentication System")

        self.landing_page()

    def landing_page(self):
        self.frame = ttk.Frame(self.master)
        self.frame.grid(row=0, column=0, padx=10, pady=10)

        ttk.Label(self.frame, text="Authentication System", font=('Arial', 16)).grid(row=0, column=0, columnspan=2, pady=10)

        ttk.Button(self.frame, text="Authenticate", command=self.start_authentication).grid(row=1, column=0, pady=10)
        ttk.Button(self.frame, text="Sign Up", command=self.show_sign_up_page).grid(row=1, column=1, pady=10)

    def start_authentication(self):
        # Your authentication logic here
        messagebox.showinfo("Authentication", "Authentication Process Started")
        self.perform_authentication()

    def perform_authentication(self):
        # Add your authentication logic here
        model = cv2.face.LBPHFaceRecognizer_create()
        model.read('faces_LBPH.yml')
        f = open('member.txt', 'r')
        names = f.readline().split(',')

        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml")

        cap = cv2.VideoCapture(0)
        cv2.namedWindow('frame',cv2.WINDOW_NORMAL)

        timenow = time.time()
        while(cap.isOpened()):
            count = 5 - int(time.time() - timenow)
            ret, img = cap.read()
            if ret == True:
                imgcopy = img.copy()
                cv2.putText(imgcopy, str(count), (200,400), cv2.FONT_HERSHEY_SIMPLEX, 15, (0,0,255), 35)
                cv2.imshow("frame", imgcopy)
                k = cv2.waitKey(100)
                if k == ord("z") or k == ord("Z") or count == 0:
                    cv2.imwrite("img02.jpeg", img)
                    break
        cap.release()
        cv2.destroyAllWindows()
        img = cv2.imread("img02.jpeg")
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1,3)
        for (x, y, w, h) in faces:
            img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 3)
            face_img = cv2.resize(gray[y: y + h, x: x + w], (400, 400))
            try:
                val = model.predict(face_img)
                if val[1] < 50:
                    print('Welcome ' + names[val[0]]  + 'Login!', val[1])
                    messagebox.showinfo("Authentication", "Authentication Successful! Welcome " + names[val[0]])
                    self.voice_recognition()
                else:
                    messagebox.showinfo("Authentication", "Sorry! You are not a member, Please sign up scan face!")
            except:
                messagebox.showinfo("Authentication", "Error during identification")
            

        # Proceed to the next steps (Voice Recognition, Text-to-Speech, Video Player)
    def talking(self, sentence, lang):
        with tempfile.NamedTemporaryFile(delete=True) as f:
            tts=gTTS(text=sentence,lang=lang)
            tts.save('{}.mp3'.format(f.name))
            mixer.music.load('{}.mp3'.format(f.name))
            mixer.music.set_endevent(pygame.USEREVENT)
            mixer.music.play(loops=0)
                
    def voice_recognition(self):
        sr_flag=True
        while True:
            try:
                for event in pygame.event.get():
                    if event.type==pygame.USEREVENT:
                        sr_flag=True
            except:
                pass
            if sr_flag==True:
                try:
                    with sr.Microphone() as source:
                        print("Would you like to open (Music or Movie):")
                        r=sr.Recognizer()
                        r.energy_threshold=4000
                        audio=r.listen(source)
                        listen_text=r.recognize_google(audio,language="en")
                        print(listen_text)
                        if "music" in listen_text:
                            self.music()
                            sr_flag=False
                        if "movie" in listen_text:
                            self.show_video_player()
                            sr_flag=False
                            break
                except sr.UnknownValueError:
                    print("Unable translate Voice")
                    sr_flag=True

    def show_video_player(self):
        # Your video player logic here
        video_player_root = tk.Toplevel(self.master)
        video_player = VideoPlayer(video_player_root)

    @staticmethod
    def saveImg(image, index, name, total):
        filename = 'images/' + name + '/face{:03d}.jpg'.format(index)
        cv2.imwrite(filename, image)
    
    def start_face_recognition(self, sign_up_window):
        name = self.name_entry.get()
        if not name:
            messagebox.showinfo("Error", "Please enter a name.")
            return

        if os.path.isdir('images/' + name):
            messagebox.showinfo("Error", "The name already exists!")
            return

        os.mkdir('images/' + name)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml")
        cap = cv2.VideoCapture(0)

        try:
            index = 1
            total = 50
            while index > 0:
                ret, frame = cap.read()
                frame = cv2.flip(frame, 1)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, 1.1, 3)
                for (x, y, w, h) in faces:
                    frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)
                    image = cv2.resize(gray[y: y + h, x: x + w], (400, 400))
                    self.saveImg(image, index, name, total)
                    sleep(0.1)
                    index += 1
                    if index > total:
                        messagebox.showinfo("Scan Complete", "Scan Complete!")
                        sign_up_window.destroy()  # Close the sign_up_window
                        index = -1
                        break
                cv2.imshow('video', frame)
                cv2.waitKey(1)
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")
        finally:
            cap.release()
            cv2.destroyAllWindows()

        images = []
        labels = []
        labelstr = []
        count = 0
        dirs = os.listdir('images')
        for d in dirs:
            if os.path.isdir('images/' + d):
                files = glob.glob('images/' + d + '/*.jpg')
                for filename in files:
                    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
                    images.append(img)
                    labels.append(count)
                labelstr.append(d)
                count += 1

        # Write labelstr to a file
        with open('member.txt', 'w') as f:
            f.write(','.join(labelstr))

        print('Start building the model...')
        model = cv2.face.LBPHFaceRecognizer_create()
        model.train(np.asarray(images), np.asarray(labels))
        model.save('faces_LBPH.yml')
        print('Model creation complete!')
    
    def show_sign_up_page(self):
        sign_up_window = tk.Toplevel(self.master)
        sign_up_window.title("Face Recognition App")

        # Styling
        sign_up_window.geometry("300x150")  # Set the initial size of the window
        sign_up_window.configure(bg="#f0f0f0")  # Set background color

        # Create and pack a label widget for the username
        label_username = tk.Label(sign_up_window, text="Enter your name:", font=("Helvetica", 12), bg="#f0f0f0")
        label_username.pack(pady=5)

        # Create and pack an entry widget for the username
        self.name_entry = tk.Entry(sign_up_window, font=("Helvetica", 12))
        self.name_entry.pack(pady=10)

        # Create and pack a button widget
        start_button = tk.Button(sign_up_window, text="Start Face Recognition", command=lambda: self.start_face_recognition(sign_up_window))
        start_button.pack()

        # Create and pack a label widget for the result
        result_label = tk.Label(sign_up_window, text="", font=("Helvetica", 12, "italic"), bg="#f0f0f0")
        result_label.pack()
        
    def music(self):
        global playsong, preplaysong, volume
        playsong = ""
        volume = 0.6

    def choose():
        msg.set("\n Now Playing : "+choice.get())
        global playsong
        playsong = choice.get()

    def pausemp3():  # Pause
        mixer.music.pause()
        msg.set("\nPause Playback {}".format(playsong))

    def increase():
        global volume
        volume += 0.1
        if volume >= 1:
            volume = 1
        mixer.music.set_volume(volume)

    def decrease():
        global volume
        volume -= 0.1
        if volume <= 0.3:
            volume = 0.3
        mixer.music.set_volume(volume)

    def playmp3():  # Play
        global status, playsong, preplaysong
        if playsong == preplaysong:  # Same Song
            if not mixer.music.get_busy():
                mixer.music.load(playsong)
                mixer.music.play(loops=-1)
            else:
                mixer.music.unpause()
            msg.set("\nNow Playing:{}".format(playsong))
        else:  # Change Song
            playNewmp3()
            preplaysong = playsong

    def playNewmp3():  # Play new song
        global playsong
        mixer.music.stop()
        mixer.music.load(playsong)
        mixer.music.play(loops=-1)
        msg.set("\nIs Playing : {}".format(playsong))

    def stopmp3():
        mixer.music.stop()
        msg.set("\n Stop Playing")

    def exitmp3():
        mixer.music.stop()
        win.destroy()
        mixer.init()
        win = tk.Tk()
        win.geometry("650x400")
        win.title("mp3 Player")

        labeltitle = tk.Label(win, text="\n mp3 Player", fg="red")
        labeltitle.pack()

        frame1 = tk.Frame(win)
        frame1.pack()

        source_dir = r"C:\Users\Mitan\Music\Songs\\"
        mp3files = glob.glob(source_dir + "*.mp3")

        playsong = preplaysong = ""
        index = 0
        volume = 0.6
        choice = tk.StringVar()

        for mp3 in mp3files:  # Created song option button
            rbtem = tk.Radiobutton(frame1, text=mp3, variable=choice, value=mp3, command=choose)
            if(index == 0):
                rbtem.select()
                playsong = preplaysong = mp3
            rbtem.grid(row=index, column=0, sticky="w")
            index += 1

        msg = tk.StringVar()
        msg.set("\nSong playing:")
        label = tk.Label(win, textvariable=msg, fg="blue", font=("System", 10))
        label.pack()
        labelsep = tk.Label(win, text="\n")
        labelsep.pack()

        frame2 = tk.Frame(win)  # Button Container
        frame2.pack()
        button1 = tk.Button(frame2, text="Play", width=8, command=playmp3)
        button1.grid(row=0, column=0, padx=5, pady=5)
        button2 = tk.Button(frame2, text="Pause", width=8, command=pausemp3)
        button2.grid(row=0, column=1, padx=5, pady=5)
        button3 = tk.Button(frame2, text="Volume Up", width=8, command=increase)
        button3.grid(row=0, column=2, padx=5, pady=5)
        button4 = tk.Button(frame2, text="Volume Down", width=8, command=decrease)
        button4.grid(row=0, column=3, padx=5, pady=5)
        button5 = tk.Button(frame2, text="Stop", width=8, command=stopmp3)
        button5.grid(row=0, column=4, padx=5, pady=5)
        button6 = tk.Button(frame2, text="Finish", width=8, command=exitmp3)
        button6.grid(row=0, column=5, padx=5, pady=5)
        win.protocol("WM_DELETE_WINDOW", exitmp3)
        win.mainloop()


    def back_to_landing_page(self):
        # Remove sign-up elements and return to the landing page
        for widget in self.frame.winfo_children():
            widget.destroy()
        self.landing_page()
        
class VideoPlayer:
    def __init__(self, master):
        self.master = master
        self.master.title("Video Player")

        self.instance = vlc.Instance()
        self.player = self.instance.media_player_new()

        self.play_button = ttk.Button(master, text="Play", command=self.play)
        self.pause_button = ttk.Button(master, text="Pause", command=self.pause)
        self.stop_button = ttk.Button(master, text="Stop", command=self.stop)

        self.play_button.grid(row=1, column=0, padx=5, pady=5)
        self.pause_button.grid(row=1, column=1, padx=5, pady=5)
        self.stop_button.grid(row=1, column=2, padx=5, pady=5)

        self.file_input = ttk.Entry(master, width=50)
        self.load_button = ttk.Button(master, text="Load Video", command=self.load_video)

        self.file_input.grid(row=0, column=0, columnspan=2, padx=5, pady=5)
        self.load_button.grid(row=0, column=2, padx=5, pady=5)

        self.is_playing = False
        self.check_playing()

    def check_playing(self):
        if self.is_playing:
            self.master.after(100, self.check_playing)
            return

        self.master.after(100, self.check_playing)

    def load_video(self):
        file_path = filedialog.askopenfilename(title="Select Video File", filetypes=[("Video files", "*.mp4;*.avi;*.mkv")])
        if file_path:
            media = self.instance.media_new(file_path)
            self.player.set_media(media)

    def play(self):
        if not self.is_playing:
            self.is_playing = True
            self.master.geometry("800x600")  # Set the default size to 800x600
            self.player.set_hwnd(self.master.winfo_id())  # Set the window handle for video output
            self.player.play()
            self.check_playing()

    def pause(self):
        self.is_playing = False
        self.player.pause()

    def stop(self):
        self.is_playing = False
        self.player.stop()

    def main():
        root = tk.Tk()
        player = VideoPlayer(root)
        display(root)
        root.mainloop()

if __name__ == "__main__":
    root = tk.Tk()
    app = AuthenticationApp(root)
    root.mainloop()

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
Welcome MitansLogin! 21.885888218420337
Would you like to open (Music or Movie):
movie
